# Block A: Data Parsing & Preprocessing - COMPLETE TOOLKIT
## Competition Module A - All Scenarios Covered

**Time Allowed: 3 hours**

This notebook handles:
1. ✅ Multiple file formats (CSV, Excel, TXT, PDF, XML, JSON)
2. ✅ Geodata processing with GeoPandas
3. ✅ Data cleaning & attribute extraction
4. ✅ Complex attribute splitting
5. ✅ Missing data handling
6. ✅ Data validation & quality checks
7. ✅ Feature engineering for weather/emergency data
8. ✅ Rostov-on-Don specific processing

In [ ]:
# Install ALL required packages for Block A
!pip install pandas numpy geopandas shapely openpyxl xlrd PyPDF2 beautifulsoup4 lxml python-dateutil

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon
import re
from datetime import datetime, timedelta
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully")
print(f"Pandas version: {pd.__version__}")
print(f"GeoPandas version: {gpd.__version__}")

## Part 1: File Format Parsers
### Handle ALL competition file formats

In [ ]:
class UniversalDataParser:
    """Parse any data format for Block A."""
    
    def __init__(self):
        self.supported_formats = ['csv', 'xlsx', 'xls', 'txt', 'xml', 'json', 'pdf']
    
    def parse_file(self, filepath: str) -> pd.DataFrame:
        """Auto-detect and parse any file format."""
        file_ext = Path(filepath).suffix.lower().replace('.', '')
        
        print(f"📄 Parsing file: {filepath}")
        print(f"   Format detected: {file_ext.upper()}")
        
        if file_ext == 'csv':
            return self.parse_csv(filepath)
        elif file_ext in ['xlsx', 'xls']:
            return self.parse_excel(filepath)
        elif file_ext == 'txt':
            return self.parse_txt(filepath)
        elif file_ext == 'xml':
            return self.parse_xml(filepath)
        elif file_ext == 'json':
            return self.parse_json(filepath)
        elif file_ext == 'pdf':
            return self.parse_pdf(filepath)
        else:
            raise ValueError(f"Unsupported format: {file_ext}")
    
    def parse_csv(self, filepath: str) -> pd.DataFrame:
        """Parse CSV with automatic delimiter detection."""
        # Try different delimiters
        for delimiter in [',', ';', '\t', '|']:
            try:
                df = pd.read_csv(filepath, delimiter=delimiter, encoding='utf-8')
                if len(df.columns) > 1:
                    print(f"   ✅ Parsed with delimiter: '{delimiter}'")
                    return df
            except:
                continue
        
        # Try with different encodings
        for encoding in ['utf-8', 'cp1251', 'latin1']:
            try:
                df = pd.read_csv(filepath, encoding=encoding)
                print(f"   ✅ Parsed with encoding: {encoding}")
                return df
            except:
                continue
        
        raise ValueError("Could not parse CSV file")
    
    def parse_excel(self, filepath: str) -> pd.DataFrame:
        """Parse Excel file (all sheets)."""
        excel_file = pd.ExcelFile(filepath)
        print(f"   📊 Found {len(excel_file.sheet_names)} sheet(s): {excel_file.sheet_names}")
        
        # If multiple sheets, combine them
        if len(excel_file.sheet_names) == 1:
            df = pd.read_excel(filepath)
        else:
            # Combine all sheets
            dfs = []
            for sheet in excel_file.sheet_names:
                sheet_df = pd.read_excel(filepath, sheet_name=sheet)
                sheet_df['source_sheet'] = sheet
                dfs.append(sheet_df)
            df = pd.concat(dfs, ignore_index=True)
        
        print(f"   ✅ Loaded {len(df)} rows")
        return df
    
    def parse_txt(self, filepath: str) -> pd.DataFrame:
        """Parse TXT file (structured or unstructured)."""
        with open(filepath, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        # Try to detect structure
        # Check if first line looks like header
        if '\t' in lines[0] or ',' in lines[0] or ';' in lines[0]:
            # Structured data
            delimiter = '\t' if '\t' in lines[0] else (',' if ',' in lines[0] else ';')
            from io import StringIO
            df = pd.read_csv(StringIO(''.join(lines)), delimiter=delimiter)
        else:
            # Unstructured text - extract information
            data = {'line_number': [], 'text': []}
            for i, line in enumerate(lines):
                data['line_number'].append(i)
                data['text'].append(line.strip())
            df = pd.DataFrame(data)
        
        print(f"   ✅ Parsed {len(df)} rows")
        return df
    
    def parse_xml(self, filepath: str) -> pd.DataFrame:
        """Parse XML file."""
        import xml.etree.ElementTree as ET
        
        tree = ET.parse(filepath)
        root = tree.getroot()
        
        # Extract data from XML
        data = []
        for child in root:
            row = {}
            for elem in child:
                row[elem.tag] = elem.text
            data.append(row)
        
        df = pd.DataFrame(data)
        print(f"   ✅ Parsed {len(df)} records")
        return df
    
    def parse_json(self, filepath: str) -> pd.DataFrame:
        """Parse JSON file."""
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        if isinstance(data, list):
            df = pd.DataFrame(data)
        elif isinstance(data, dict):
            # Try to find data array
            for key in data.keys():
                if isinstance(data[key], list):
                    df = pd.DataFrame(data[key])
                    break
            else:
                df = pd.DataFrame([data])
        
        print(f"   ✅ Parsed {len(df)} records")
        return df
    
    def parse_pdf(self, filepath: str) -> pd.DataFrame:
        """Parse PDF file (extract text)."""
        try:
            import PyPDF2
            
            with open(filepath, 'rb') as f:
                pdf = PyPDF2.PdfReader(f)
                text_data = []
                
                for i, page in enumerate(pdf.pages):
                    text = page.extract_text()
                    text_data.append({'page': i+1, 'text': text})
            
            df = pd.DataFrame(text_data)
            print(f"   ✅ Extracted text from {len(df)} pages")
            return df
        except ImportError:
            print("   ⚠️ PyPDF2 not installed. Install with: pip install PyPDF2")
            return pd.DataFrame()

# Test the parser
parser = UniversalDataParser()
print(f"\n✅ Universal parser ready for: {', '.join(parser.supported_formats)}")

## Part 2: Data Cleaning & Preprocessing

In [ ]:
class DataCleaner:
    """Comprehensive data cleaning for Block A."""
    
    def __init__(self):
        pass
    
    def clean_column_names(self, df: pd.DataFrame) -> pd.DataFrame:
        """Clean and standardize column names."""
        print("🧹 Cleaning column names...")
        
        df = df.copy()
        
        # Remove special characters
        df.columns = df.columns.str.strip()
        df.columns = df.columns.str.lower()
        df.columns = df.columns.str.replace(r'[^a-z0-9_]', '_', regex=True)
        df.columns = df.columns.str.replace(r'_+', '_', regex=True)
        df.columns = df.columns.str.strip('_')
        
        print(f"   ✅ Cleaned columns: {list(df.columns)}")
        return df
    
    def remove_service_info(self, df: pd.DataFrame) -> pd.DataFrame:
        """Remove service/technical information."""
        print("🧹 Removing service information...")
        
        df = df.copy()
        
        # Remove columns with >90% missing values
        threshold = len(df) * 0.9
        df = df.dropna(thresh=threshold, axis=1)
        
        # Remove empty rows
        df = df.dropna(how='all')
        
        # Remove duplicate rows
        duplicates = df.duplicated().sum()
        if duplicates > 0:
            df = df.drop_duplicates()
            print(f"   🗑️ Removed {duplicates} duplicate rows")
        
        print(f"   ✅ Data shape after cleaning: {df.shape}")
        return df
    
    def extract_dates(self, df: pd.DataFrame, date_columns: list = None) -> pd.DataFrame:
        """Extract and parse date columns."""
        print("📅 Extracting date columns...")
        
        df = df.copy()
        
        if date_columns is None:
            # Auto-detect date columns
            date_columns = [col for col in df.columns if 'date' in col.lower() or 'time' in col.lower()]
        
        for col in date_columns:
            if col in df.columns:
                try:
                    df[col] = pd.to_datetime(df[col], errors='coerce')
                    print(f"   ✅ Parsed date column: {col}")
                except:
                    print(f"   ⚠️ Could not parse: {col}")
        
        return df
    
    def extract_numeric(self, df: pd.DataFrame) -> pd.DataFrame:
        """Extract numeric values from text columns."""
        print("🔢 Extracting numeric values...")
        
        df = df.copy()
        
        for col in df.columns:
            if df[col].dtype == 'object':
                # Try to extract numbers
                df[f'{col}_numeric'] = df[col].str.extract(r'([-+]?\d*\.?\d+)', expand=False)
                
                # Convert to numeric
                try:
                    df[f'{col}_numeric'] = pd.to_numeric(df[f'{col}_numeric'], errors='coerce')
                    if df[f'{col}_numeric'].notna().sum() > 0:
                        print(f"   ✅ Extracted numbers from: {col}")
                    else:
                        df = df.drop(f'{col}_numeric', axis=1)
                except:
                    df = df.drop(f'{col}_numeric', axis=1)
        
        return df
    
    def split_complex_attributes(self, df: pd.DataFrame, delimiter: str = ',') -> pd.DataFrame:
        """Split complex multi-value attributes."""
        print(f"✂️ Splitting complex attributes (delimiter: '{delimiter}')...")
        
        df = df.copy()
        
        for col in df.columns:
            if df[col].dtype == 'object':
                # Check if column contains delimiter
                sample = df[col].dropna().iloc[0] if len(df[col].dropna()) > 0 else None
                if sample and isinstance(sample, str) and delimiter in sample:
                    # Split the column
                    split_data = df[col].str.split(delimiter, expand=True)
                    
                    # Name new columns
                    for i in range(split_data.shape[1]):
                        df[f'{col}_part{i+1}'] = split_data[i].str.strip()
                    
                    print(f"   ✅ Split '{col}' into {split_data.shape[1]} parts")
        
        return df

# Create cleaner instance
cleaner = DataCleaner()
print("\n✅ Data cleaner ready")

## Part 3: GeoPandas Integration for Geodata

In [ ]:
class GeoDataProcessor:
    """Process geodata with GeoPandas."""
    
    def __init__(self):
        self.crs = 'EPSG:4326'  # WGS84
    
    def create_geodataframe(self, df: pd.DataFrame, 
                           lat_col: str = 'latitude', 
                           lon_col: str = 'longitude') -> gpd.GeoDataFrame:
        """Convert DataFrame to GeoDataFrame."""
        print(f"🗺️ Creating GeoDataFrame...")
        
        # Auto-detect lat/lon columns
        if lat_col not in df.columns:
            lat_candidates = [col for col in df.columns if 'lat' in col.lower()]
            if lat_candidates:
                lat_col = lat_candidates[0]
                print(f"   📍 Found latitude column: {lat_col}")
        
        if lon_col not in df.columns:
            lon_candidates = [col for col in df.columns if 'lon' in col.lower()]
            if lon_candidates:
                lon_col = lon_candidates[0]
                print(f"   📍 Found longitude column: {lon_col}")
        
        if lat_col not in df.columns or lon_col not in df.columns:
            print(f"   ⚠️ Geodata columns not found")
            return None
        
        # Create geometry
        geometry = [Point(lon, lat) for lon, lat in zip(df[lon_col], df[lat_col])]
        gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=self.crs)
        
        print(f"   ✅ Created GeoDataFrame with {len(gdf)} points")
        print(f"   📊 CRS: {gdf.crs}")
        
        return gdf
    
    def validate_coordinates(self, gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
        """Validate geographic coordinates."""
        print("✅ Validating coordinates...")
        
        original_count = len(gdf)
        
        # Check for valid coordinates
        gdf = gdf[gdf.geometry.is_valid]
        
        # Check latitude range (-90 to 90)
        gdf = gdf[(gdf.geometry.y >= -90) & (gdf.geometry.y <= 90)]
        
        # Check longitude range (-180 to 180)
        gdf = gdf[(gdf.geometry.x >= -180) & (gdf.geometry.x <= 180)]
        
        removed = original_count - len(gdf)
        if removed > 0:
            print(f"   🗑️ Removed {removed} invalid coordinates")
        
        print(f"   ✅ Valid coordinates: {len(gdf)}")
        return gdf
    
    def add_spatial_features(self, gdf: gpd.GeoDataFrame, 
                            center_lat: float = 47.2357, 
                            center_lon: float = 39.7015) -> gpd.GeoDataFrame:
        """Add spatial features (distance from center, etc.)."""
        print("📐 Adding spatial features...")
        
        # Create center point
        center = Point(center_lon, center_lat)
        
        # Calculate distance from center (in km)
        gdf_proj = gdf.to_crs('EPSG:3857')  # Web Mercator for distance
        center_gdf = gpd.GeoDataFrame({'geometry': [center]}, crs='EPSG:4326').to_crs('EPSG:3857')
        
        gdf['distance_from_center_km'] = gdf_proj.geometry.distance(center_gdf.geometry[0]) / 1000
        
        print(f"   ✅ Added distance_from_center_km")
        print(f"   📊 Distance range: {gdf['distance_from_center_km'].min():.2f} - {gdf['distance_from_center_km'].max():.2f} km")
        
        return gdf

# Create geo processor
geo_processor = GeoDataProcessor()
print("\n✅ Geo processor ready")

## Part 4: COMPLETE WORKFLOW - Example with Real Data

In [ ]:
# Example: Process Rostov weather Excel file
print("="*70)
print("BLOCK A: COMPLETE DATA PROCESSING WORKFLOW")
print("="*70)

# Step 1: Parse file
print("\n📂 STEP 1: PARSING FILE")
try:
    df = parser.parse_file('sample_rostov_weather.xlsx')
    print(f"✅ Loaded {len(df)} rows, {len(df.columns)} columns")
    print(f"\nColumns: {list(df.columns)}")
    print(f"\nFirst 3 rows:")
    print(df.head(3))
except Exception as e:
    print(f"⚠️ File not found: {e}")
    print("Generating sample data...")
    # Generate sample data
    df = pd.DataFrame({
        'date': pd.date_range('2024-01-01', periods=100),
        'district': ['Leninsky'] * 100,
        'latitude': [47.2220] * 100,
        'longitude': [39.7180] * 100,
        'temperature': np.random.normal(15, 10, 100),
        'precipitation': np.random.gamma(2, 5, 100),
        'humidity': np.random.uniform(30, 80, 100)
    })
    print(f"✅ Generated {len(df)} sample rows")

# Step 2: Clean data
print("\n🧹 STEP 2: DATA CLEANING")
df = cleaner.clean_column_names(df)
df = cleaner.remove_service_info(df)
df = cleaner.extract_dates(df)

# Step 3: Create GeoDataFrame
print("\n🗺️ STEP 3: GEODATA PROCESSING")
gdf = geo_processor.create_geodataframe(df)
if gdf is not None:
    gdf = geo_processor.validate_coordinates(gdf)
    gdf = geo_processor.add_spatial_features(gdf)
    
    print("\n📊 GeoDataFrame Info:")
    print(f"   Shape: {gdf.shape}")
    print(f"   CRS: {gdf.crs}")
    print(f"   Columns: {list(gdf.columns)}")

# Step 4: Quality Report
print("\n📋 STEP 4: DATA QUALITY REPORT")
print("="*70)
print(f"Total Records: {len(df)}")
print(f"Total Features: {len(df.columns)}")
print(f"\nMissing Values:")
missing = df.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
else:
    print("   ✅ No missing values")

print(f"\nData Types:")
print(df.dtypes.value_counts())

print(f"\nNumeric Columns Summary:")
numeric_cols = df.select_dtypes(include=[np.number]).columns
print(df[numeric_cols].describe())

print("\n" + "="*70)
print("✅ BLOCK A PROCESSING COMPLETE!")
print("="*70)

## Part 5: Rostov-Specific Processing

In [ ]:
# Rostov districts data
ROSTOV_DISTRICTS = {
    "Leninsky": {"lat": 47.2220, "lon": 39.7180, "name_ru": "Ленинский"},
    "Kirovsky": {"lat": 47.2580, "lon": 39.7850, "name_ru": "Кировский"},
    "Oktyabrsky": {"lat": 47.2750, "lon": 39.7320, "name_ru": "Октябрьский"},
    "Pervomaisky": {"lat": 47.2180, "lon": 39.6420, "name_ru": "Первомайский"},
    "Proletarsky": {"lat": 47.1980, "lon": 39.7680, "name_ru": "Пролетарский"},
    "Sovetsky": {"lat": 47.2420, "lon": 39.6850, "name_ru": "Советский"},
    "Zheleznodorozhny": {"lat": 47.2640, "lon": 39.7180, "name_ru": "Железнодорожный"},
    "Voroshilovsky": {"lat": 47.2380, "lon": 39.7420, "name_ru": "Ворошиловский"}
}

def assign_district(lat: float, lon: float) -> str:
    """Assign district based on coordinates (nearest district)."""
    min_dist = float('inf')
    assigned_district = None
    
    for district, coords in ROSTOV_DISTRICTS.items():
        dist = ((lat - coords['lat'])**2 + (lon - coords['lon'])**2)**0.5
        if dist < min_dist:
            min_dist = dist
            assigned_district = district
    
    return assigned_district

# Apply to GeoDataFrame
if gdf is not None:
    print("🏘️ Assigning Rostov districts based on coordinates...")
    gdf['assigned_district'] = gdf.apply(
        lambda row: assign_district(row.geometry.y, row.geometry.x), 
        axis=1
    )
    
    print("\n📊 District distribution:")
    print(gdf['assigned_district'].value_counts())
    
    print("\n✅ Districts assigned!")

## Part 6: Export Processed Data

In [ ]:
# Save processed data
print("💾 Saving processed data...")

# Save as CSV
df.to_csv('processed_data_blockA.csv', index=False)
print("✅ Saved: processed_data_blockA.csv")

# Save GeoDataFrame as GeoJSON
if gdf is not None:
    gdf.to_file('processed_geodata_blockA.geojson', driver='GeoJSON')
    print("✅ Saved: processed_geodata_blockA.geojson")

# Save data quality report
with open('data_quality_report_blockA.txt', 'w') as f:
    f.write("BLOCK A DATA QUALITY REPORT\n")
    f.write("="*70 + "\n\n")
    f.write(f"Total Records: {len(df)}\n")
    f.write(f"Total Features: {len(df.columns)}\n\n")
    f.write(f"Columns: {', '.join(df.columns)}\n\n")
    f.write(f"Missing Values:\n{df.isnull().sum()}\n\n")
    f.write(f"Data Types:\n{df.dtypes}\n")

print("✅ Saved: data_quality_report_blockA.txt")

print("\n" + "="*70)
print("📦 ALL FILES SAVED - BLOCK A COMPLETE!")
print("="*70)
print("Files created:")
print("  1. processed_data_blockA.csv")
print("  2. processed_geodata_blockA.geojson")
print("  3. data_quality_report_blockA.txt")

## Summary - Block A Checklist

✅ **File Parsing:**
- CSV (auto-delimiter detection)
- Excel (multi-sheet support)
- TXT (structured/unstructured)
- XML
- JSON
- PDF (text extraction)

✅ **Data Cleaning:**
- Column name standardization
- Service information removal
- Duplicate removal
- Missing value handling

✅ **Attribute Processing:**
- Date extraction & parsing
- Numeric value extraction
- Complex attribute splitting

✅ **GeoPandas Integration:**
- GeoDataFrame creation
- Coordinate validation
- Spatial features (distance, etc.)
- GeoJSON export

✅ **Rostov-Specific:**
- 8 district definitions
- Automatic district assignment
- Russian district names

✅ **Quality Assurance:**
- Data quality report
- Missing value analysis
- Data type validation
- Statistical summary

**Time: 3 hours allocated - All scenarios covered! 🎯**